In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.ensemble import RandomForestRegressor as RFR

In [2]:
dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}
train =pd.read_csv('E://Favorita/train/train.csv', dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 106672217))
test =pd.read_csv('E://Favorita/test.csv/test.csv', dtype=dtypes, parse_dates=['date'])

In [13]:
items=pd.read_csv('E://Favorita/items.csv/items.csv')

In [3]:
#removing all observation from training dataset which doesnt exist in test
test_unique=test['item_nbr'].unique().tolist()
train_unique =pd.DataFrame(data=train['item_nbr'].unique(), columns={'item_num'})
train_unique['isin'] =train_unique.isin(test_unique)
to_remove=train_unique[(train_unique[['isin']]==False).values].item_num.values
train_index = train[['item_nbr']].isin(to_remove).values
train_short=train[~train_index]
train_short['wd']=train_short['date'].dt.dayofweek

C:\Users\woj\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
avg_wd = train_short[['date','wd','store_nbr','item_nbr','unit_sales','id']].groupby(['wd','item_nbr','store_nbr'])['unit_sales'].mean().to_frame('avg_wday').reset_index()
avg_full = avg_wd.groupby(['item_nbr', 'store_nbr'])['avg_wday'].mean().to_frame('avg_day').reset_index()

In [5]:
newnew=pd.merge(train_short, avg_full,how='left', on=['item_nbr', 'store_nbr'])
newnew=pd.merge(newnew, avg_wd,how='left', on=['item_nbr', 'store_nbr','wd'])
newnew['onpromotion'].fillna('False')
del train, train_short

In [26]:
sum(pd.isnull(newnew['onpromotion']))

0

In [85]:
newnew.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,wd,avg_day,avg_wday,avg_class,family,class,perishable,avg_class_store,0
0,106672216,2017-02-18,34,463901,1.0,False,5,1.919471,1.863636,4.676555,CLEANING,3034,0,2.980253,37666
1,106672217,2017-02-18,34,463903,2.0,False,5,1.453537,1.375000,4.676555,CLEANING,3034,0,2.980253,37666
2,106672218,2017-02-18,34,464263,2.0,False,5,5.300080,5.958333,11.137202,GROCERY I,1014,0,9.238782,116155
3,106672219,2017-02-18,34,464302,3.0,False,5,1.726367,2.200000,4.548905,GROCERY I,1026,0,3.869122,116155
4,106672220,2017-02-18,34,464333,26.0,False,5,13.628486,17.799999,8.787640,GROCERY I,1040,0,11.485539,116155


In [9]:
#preparing test data
test['wd']=test['date'].dt.dayofweek
test=pd.merge(test, avg_full,how='left', on=['item_nbr', 'store_nbr'])
test=pd.merge(test, avg_wd,how='left', on=['item_nbr', 'store_nbr','wd'])

In [62]:
newnew=pd.merge(newnew, items,how='left', on=['item_nbr'])

In [61]:
items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [66]:
avg_class_full=newnew.groupby(['class'])['unit_sales'].mean().to_frame('avg_class').reset_index()
avg_class_store = newnew.groupby(['class','store_nbr'])['unit_sales'].mean().to_frame('avg_class_store').reset_index()
avg_trans_family = newnew.groupby( ['family','store_nbr']).size().to_frame('avg_trans_family').reset_index()

In [83]:
avg_trans_family = newnew.groupby( ['family','store_nbr']).size().to_frame('avg_trans_family').reset_index()

In [69]:
newnew=pd.merge(newnew, avg_class_full, how='left', on=['class'])
newnew=pd.merge(newnew, avg_class_store, how='left', on=['class','store_nbr'])
newnew=pd.merge(newnew, avg_trans_family, how='left', on=['family','store_nbr'])

In [84]:
newnew.drop('0', 1, inplace=True)

ValueError: labels ['0'] not contained in axis

In [81]:
newnew.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,wd,avg_day,avg_wday,avg_class,family,class,perishable,avg_class_store,0
0,106672216,2017-02-18,34,463901,1.0,False,5,1.919471,1.863636,4.676555,CLEANING,3034,0,2.980253,37666
1,106672217,2017-02-18,34,463903,2.0,False,5,1.453537,1.375000,4.676555,CLEANING,3034,0,2.980253,37666
2,106672218,2017-02-18,34,464263,2.0,False,5,5.300080,5.958333,11.137202,GROCERY I,1014,0,9.238782,116155
3,106672219,2017-02-18,34,464302,3.0,False,5,1.726367,2.200000,4.548905,GROCERY I,1026,0,3.869122,116155
4,106672220,2017-02-18,34,464333,26.0,False,5,13.628486,17.799999,8.787640,GROCERY I,1040,0,11.485539,116155


In [71]:
del avg_class_full, avg_class_store